In [1]:
import tensorflow as tf
import numpy as np
from custom_layers import *
from worker import Worker

In [2]:
coordinator = tf.train.Coordinator()

hidden_layer_size = 128
output_size = 10
global_model = FullyConnectedModel(
                output_size=output_size,
                hidden_layer_size=hidden_layer_size,
            )
#_ = global_model([global_model.inputs['img'], global_model.inputs['mask']])
#print(global_model.accumulators)
fake_img = np.zeros((1, 28, 28))
fake_mask = np.zeros((1, hidden_layer_size))
print("Init call: ", global_model([fake_img, fake_mask]))
'''
for layer in global_model.layers:
    layer.trainable = False
'''
global_model.summary()
print("weights: ", global_model.get_weights())
print("accumulators: ", global_model.accumulators)
global_model.init_accumulators()
print("accumulators: ", global_model.accumulators)
print("weights: ", global_model.get_weights())
print("trainable: ", global_model.non_trainable_weights)
#global_model.copy_weights(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Init call:  tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)
Model: "fully_connected_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_flatte multiple                  0         
_________________________________________________________________
fully_connected_model_encode multiple                  100480    
____________________________________________________________

In [3]:
global_model.trainable_weights

[<tf.Variable 'fully_connected_model/fully_connected_model_encoder/kernel:0' shape=(784, 128) dtype=float32, numpy=
 array([[ 0.01358852,  0.00920401, -0.0630241 , ..., -0.04182381,
          0.01747631, -0.06552696],
        [ 0.06142623,  0.027003  ,  0.05231097, ..., -0.01128632,
          0.02570017,  0.07943635],
        [-0.03559585, -0.01558601,  0.0322796 , ..., -0.05504216,
          0.06375351,  0.01744417],
        ...,
        [ 0.02798843,  0.05507424,  0.00544558, ...,  0.07138587,
         -0.06886146,  0.0065852 ],
        [ 0.05121624, -0.06960282,  0.06113014, ..., -0.0367342 ,
          0.06711639,  0.03394403],
        [-0.07553574,  0.00420285,  0.05101948, ..., -0.05476599,
          0.00701997,  0.01660646]], dtype=float32)>,
 <tf.Variable 'fully_connected_model/fully_connected_model_encoder/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [4]:
num_models = 10
models = {}
for idx in range(num_models):
    model = FullyConnectedModel(
            output_size=output_size,
            hidden_layer_size=hidden_layer_size,
            #parent_model=global_model,
        )
    #_ = model([model.inputs['img'], model.inputs['mask']])
    print(model([fake_img, fake_mask]))
    models[idx] = model
    #model.set_parent_model(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting

In [5]:
for idx in range(num_models):
    t = Worker(args=idx, kwargs=
                {
                    'coordinator': coordinator,
                    'global_model': global_model,
                    'model': models[idx],
                    'hidden_layer_size': hidden_layer_size,
                    'output_size': output_size,
                    'learning_rate': 1e-3,
                    'dropout_rate': 1-0.2,
                    'batch_size': 64,
                    'epochs': 20,
                    'update_batches': 1,
                    'regenerate_masks': False,
                    'mask_generation_batches': 1000
                }
    )
    t.start()


Model: "fully_connected_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_1_flat multiple                  0         
_________________________________________________________________
fully_connected_model_1_enco multiple                  100480    
_________________________________________________________________
fully_connected_model_1_Cust multiple                  0         
_________________________________________________________________
fully_connected_model_1_outp multiple                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
None
Model: "fully_connected_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_2_flat multiple                  0         

np.sum(mask):  102.0
mask:  [[0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.
  0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1.
  1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1.
  1. 1. 0. 1. 1. 1. 1. 1.]]
np.sum(mask):  89.0
mask:  [[1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
  1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1.
  1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1.
  0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 0. 0. 1. 1. 1. 1. 0.]]
x_train.shape:  (60000, 28, 28)
y_train.shape:  (60000, 10)
masks_train.shape:  (60000, 128)
x_valid.shape:  (10000, 28, 28)
y_valid.shape:  (10000, 

Epoch: 0	Thread: 6	step 0	mean loss = tf.Tensor(0.20916322, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.046875, shape=(), dtype=float32)
Epoch: 0	Thread: 8	step 0	mean loss = tf.Tensor(0.19493034, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.09375, shape=(), dtype=float32)
Epoch: 0	Thread: 7	step 0	mean loss = tf.Tensor(0.18977757, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.09375, shape=(), dtype=float32)
Epoch: 0	Thread: 9	step 0	mean loss = tf.Tensor(0.18584092, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.03125, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 100	mean loss = tf.Tensor(0.039045364, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8341584, shape=(), dtype=float32)
Epoch: 0	Thread: 5	step 100	mean loss = tf.Tensor(0.03666457, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.83586013, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 100	mean loss = tf.Tensor(0.037400004, shape=(), dtype=float32)
	mean accuracy = tf.Tenso

Epoch: 0	Thread: 3	step 600	mean loss = tf.Tensor(0.020836707, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.921901, shape=(), dtype=float32)
Epoch: 0	Thread: 2	step 600	mean loss = tf.Tensor(0.0218598, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.91755927, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 600	mean loss = tf.Tensor(0.02060473, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9230709, shape=(), dtype=float32)
Epoch: 0	Thread: 5	step 600	mean loss = tf.Tensor(0.020482192, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9206791, shape=(), dtype=float32)
Epoch: 0	Thread: 4	step 600	mean loss = tf.Tensor(0.020109147, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.92728263, shape=(), dtype=float32)
Epoch: 0	Thread: 7	step 600	mean loss = tf.Tensor(0.022306552, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9225249, shape=(), dtype=float32)
Epoch: 0	Thread: 9	step 600	mean loss = tf.Tensor(0.022053212, shape=(), dtype=float32)
	mean ac

Epoch: 1	Thread: 7	step 100	mean loss = tf.Tensor(0.019214248, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9348369, shape=(), dtype=float32)
Epoch: 1	Thread: 4	step 100	mean loss = tf.Tensor(0.017190633, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9412614, shape=(), dtype=float32)
Epoch: 1	Thread: 6	step 100	mean loss = tf.Tensor(0.017811932, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9369433, shape=(), dtype=float32)
Epoch: 1	Thread: 9	step 100	mean loss = tf.Tensor(0.019174613, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9378912, shape=(), dtype=float32)
Epoch: 1	Thread: 8	step 100	mean loss = tf.Tensor(0.017747337, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.94235975, shape=(), dtype=float32)
Epoch: 1	Thread: 0	step 200	mean loss = tf.Tensor(0.018335754, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.93582565, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 200	mean loss = tf.Tensor(0.017664142, shape=(), dtype=float32)
	mea

...

Epoch: 17	Thread: 1	step 300	mean loss = tf.Tensor(0.01053678, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9709342, shape=(), dtype=float32)
Epoch: 17	Thread: 8	step 300	mean loss = tf.Tensor(0.010390236, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9749169, shape=(), dtype=float32)
Epoch: 17	Thread: 0	step 400	mean loss = tf.Tensor(0.012550542, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.962202, shape=(), dtype=float32)
Epoch: 17	Thread: 5	step 400	mean loss = tf.Tensor(0.010705248, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9717863, shape=(), dtype=float32)
Epoch: 17	Thread: 4	step 400	mean loss = tf.Tensor(0.01041105, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9735249, shape=(), dtype=float32)
Epoch: 17	Thread: 3	step 400	mean loss = tf.Tensor(0.011860525, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.96754885, shape=(), dtype=float32)
Epoch: 17	Thread: 7	step 400	mean loss = tf.Tensor(0.011042187, shape=(), dtype=float32)
	

Epoch: 17	Thread: 3	step 900	mean loss = tf.Tensor(0.011872415, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.96736276, shape=(), dtype=float32)
Start of epoch 18
Epoch: 18	Thread: 0	step 0	mean loss = tf.Tensor(0.01253754, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9622346, shape=(), dtype=float32)
Epoch: 17	Thread: 6	step 900	mean loss = tf.Tensor(0.010682972, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9694914, shape=(), dtype=float32)
Epoch: 17	Thread: 7	step 900	mean loss = tf.Tensor(0.01099888, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9701929, shape=(), dtype=float32)
Epoch: 17	Thread: 2	step 900	mean loss = tf.Tensor(0.010985466, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9675381, shape=(), dtype=float32)
Epoch: 17	Thread: 9	step 900	mean loss = tf.Tensor(0.01075538, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9718187, shape=(), dtype=float32)
Epoch: 17	Thread: 1	step 900	mean loss = tf.Tensor(0.010473725, shape=(), 

Epoch: 18	Thread: 2	step 400	mean loss = tf.Tensor(0.010940359, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9677171, shape=(), dtype=float32)
Epoch: 18	Thread: 9	step 400	mean loss = tf.Tensor(0.010709794, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.971977, shape=(), dtype=float32)
Epoch: 18	Thread: 1	step 400	mean loss = tf.Tensor(0.01042299, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.97141445, shape=(), dtype=float32)
Epoch: 18	Thread: 8	step 400	mean loss = tf.Tensor(0.010314848, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.97526556, shape=(), dtype=float32)
Epoch: 18	Thread: 0	step 500	mean loss = tf.Tensor(0.012539751, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.96212536, shape=(), dtype=float32)
Epoch: 18	Thread: 4	step 500	mean loss = tf.Tensor(0.010340051, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9738612, shape=(), dtype=float32)
Epoch: 18	Thread: 5	step 500	mean loss = tf.Tensor(0.01062781, shape=(), dtype=float32)

Epoch: 18	Thread: 8	step 900	mean loss = tf.Tensor(0.01028572, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9754128, shape=(), dtype=float32)
Start of epoch 19
Start of epoch 19
Epoch: 19	Thread: 5	step 0	mean loss = tf.Tensor(0.01059481, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.97237784, shape=(), dtype=float32)
Epoch: 19	Thread: 4	step 0	mean loss = tf.Tensor(0.010304951, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9740383, shape=(), dtype=float32)
Start of epoch 19
Epoch: 19	Thread: 3	step 0	mean loss = tf.Tensor(0.011889558, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.96693605, shape=(), dtype=float32)
Start of epoch 19
Epoch: 19	Thread: 6	step 0	mean loss = tf.Tensor(0.010616871, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9697973, shape=(), dtype=float32)
Start of epoch 19
Start of epoch 19
Epoch: 19	Thread: 2	step 0	mean loss = tf.Tensor(0.010890629, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.967943, shape=(), dtype=

Epoch: 19	Thread: 3	step 500	mean loss = tf.Tensor(0.011893211, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.96676975, shape=(), dtype=float32)
Epoch: 19	Thread: 6	step 500	mean loss = tf.Tensor(0.010587169, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.96992314, shape=(), dtype=float32)
Epoch: 19	Thread: 2	step 500	mean loss = tf.Tensor(0.010852432, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9680768, shape=(), dtype=float32)
Epoch: 19	Thread: 7	step 500	mean loss = tf.Tensor(0.010870354, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9707166, shape=(), dtype=float32)
Epoch: 19	Thread: 9	step 500	mean loss = tf.Tensor(0.010610591, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9722848, shape=(), dtype=float32)
Epoch: 19	Thread: 1	step 500	mean loss = tf.Tensor(0.010316454, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.9718744, shape=(), dtype=float32)
Epoch: 19	Thread: 8	step 500	mean loss = tf.Tensor(0.010247817, shape=(), dtype=float3